# Gallery

Example plots created with `outset`.

Contributions welcome!
Open [an issue](https://github.com/mmore500/outset/issues) or [a pull request](https://github.com/mmore500/outset/pulls).

## Table of Contents

- [`cartopy`](#cartopy)
- [`heatmap`](#heatmap)
- [`imshow`](#imshow)
- [`kdeplot`](#kdeplot)
- [`lineplot`](#lineplot)
- [`heatplot`](#heatplot)
- [`plot`](#plot)
- [`regplot`](#regplot)
- [`scatterplot`](#scatterplot)
- [`streamplot`](#streamplot)


## Setup

In [ ]:
import copy

import cartopy.crs as ccrs
from matplotlib import cbook as mpl_cbook
from matplotlib import cm as mpl_cm
from matplotlib import colors as mpl_colors
from matplotlib import patches as mpl_patches
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import outset as otst
from outset import mark as otst_mark
from outset import patched as otst_patched
from outset import tweak as otst_tweak
from outset import util as otst_util


# mpl.rcParams["patch.linewidth"] = 0  # disable legend borders

## `cartopy`

In [ ]:
url = "https://map1c.vis.earthdata.nasa.gov/wmts-geo/wmts.cgi"
layer = "VIIRS_CityLights_2012"


og = otst.OutsetGrid(
    otst_util.NamedFrames(
        pasta=(12, 40, 16, 42),
        croissant=(1, 48, 3, 50),
    ),
    col="dish",
    marqueeplot_kws={"leader_face_kws": {"alpha": 0}},
    palette=sns.color_palette("husl", 2),
    subplot_kws={"projection": ccrs.PlateCarree()},
)


def plot(ax):
    ax.add_wmts(url, layer)
    ax.set_extent([-15, 25, 35, 60], crs=ccrs.PlateCarree())


og.broadcast(plot)
og.marqueeplot()

pass

## `heatmap`

In [ ]:
# Generate random data
data = np.random.randn(10000, 2)
x, y = data[:, 0], data[:, 1]

# Create a 2D histogram
heatmap, xedges, yedges = np.histogram2d(x, y, bins=(100, 100))

og = otst.OutsetGrid(
    [(50.5, 53.5, 56.5, 60.5)],
    aspect=1,
    color="fuchsia",
    marqueeplot_outset_kws={"frame_edge_kws": {"lw": 4, "ec": "pink"}},
    marqueeplot_source_kws={"leader_stretch": 1},
)

og.broadcast(
    sns.heatmap,
    heatmap.T,
    cmap="jet",
    cbar=False,
    square=True,
    vmin=0,
    vmax=np.max(heatmap.flat),
    zorder=-2,
)

for ax in og.axes.flat:
    ax.invert_yaxis()

og.marqueeplot()
cbar = og.figure.colorbar(
    mpl_cm.ScalarMappable(
        norm=mpl_colors.Normalize(vmin=0, vmax=np.max(heatmap.flat)),
        cmap="jet",
    ),
    ax=og.axes.flat[-1],
    location="right",
    fraction=1e-9,
)
cbar.ax.set_position([1, 0.15, 0.2, 0.73])

for ax in og.axes.flat:
    ax.xaxis.set_major_locator(plt.AutoLocator())
    ax.xaxis.set_major_formatter(plt.ScalarFormatter())
    ax.yaxis.set_major_locator(plt.AutoLocator())
    ax.yaxis.set_major_formatter(plt.ScalarFormatter())

In [ ]:
# Gener8ate random data
data = np.random.gumbel(size=(10000, 2))
x, y = data[:, 0], data[:, 1]

# Create a 2D histogram
heatmap, xedges, yedges = np.histogram2d(x, y, bins=(100, 100))

og = otst.OutsetGrid(
    [(9.5, 16.5, 25.5, 23.5)],
    # aspect=1,
    color="red",
    marqueeplot_kws={
        "color": "fuchsia",
        "mark_glyph_kws": {"color": "fuchsia"},
        "frame_edge_kws": {"lw": 3},
    },
    marqueeplot_source_kws={"leader_stretch": 0.6},
)

og.broadcast(
    sns.heatmap,
    heatmap,
    cmap="jet",
    cbar=False,
    square=True,
    zorder=-1,
)


for ax in og.axes.flat:
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)


otst.inset_outsets(
    og,
    insets="NE",
    strip_spines=False,
    strip_ticks=False,
    equalize_aspect=False,
)
for ax in og.outset_axes:
    ax.set_aspect("auto")
    ax.set_autoscale_on(True)

og.marqueeplot()

for ax in og.outset_axes:
    ax.tick_params(color="white", labelcolor="white")
    for spine in ax.spines.values():
        spine.set_linewidth(2)
        spine.set_edgecolor("white")
        spine.set_visible(True)
    sns.despine(ax=ax)


pass

## `imshow`

In [ ]:
with mpl_cbook.get_sample_data("grace_hopper.jpg") as image_file:
    image = plt.imread(image_file)

og = otst.OutsetGrid(
    data=otst_util.NamedFrames(
        {
            "hat": (0.42, 0.78, 0.62, 0.98),
            "badge": (0.10, 0.14, 0.40, 0.21),
        }
    ),
    aspect=0.9,
    col="swag",
    hue="swag",
)
og.broadcast(lambda: plt.axis("off"))
og.broadcast(plt.imshow, image, extent=(0, 1, 0, 1), origin="upper", zorder=-1)
og.marqueeplot()
og.source_axes.set_title("The Hopster", loc="left")

pass

In [ ]:
with mpl_cbook.get_sample_data("grace_hopper.jpg") as image_file:
    image = plt.imread(image_file)

og = otst.OutsetGrid(
    data=otst_util.NamedFrames(
        {
            "hat": (0.42, 0.78, 0.62, 0.98),
            "badge": (0.10, 0.14, 0.40, 0.21),
        }
    ),
    aspect=0.9,
    col="swag",
    hue="swag",
    marqueeplot_kws={
        "mark_glyph": otst_mark.MarkMagnifyingGlass(),
        "mark_glyph_kws": {"markersize": 25},
    },
    include_sourceplot=False,
)
og.broadcast(lambda: plt.axis("off"))
og.broadcast(plt.imshow, image, extent=(0, 1, 0, 1), origin="upper", zorder=-1)
og.marqueeplot(equalize_aspect=False, preserve_aspect=True)
og.add_legend()
og.set_titles("")

pass

In [ ]:
with mpl_cbook.get_sample_data("grace_hopper.jpg") as image_file:
    image = plt.imread(image_file)

og = otst.OutsetGrid(
    data=[(0.42, 0.78, 0.62, 0.98), (0.10, 0.14, 0.40, 0.21)],
    aspect=0.9,
    height=5,
    marqueeplot_kws={
        "frame_inner_pad": 0.0,
        "frame_outer_pad": 0.05,
        "frame_outer_pad_unit": "inches",
        "frame_edge_kws": {"linewidth": 4, "linestyle": "-"},
        "leader_edge_kws": {"linewidth": 2, "linestyle": "-"},
        "leader_face_kws": {"alpha": 1.0},
        "mark_glyph_kws": {"markersize": 20},
    },
    marqueeplot_source_kws={
        "leader_stretch": 0.4,
        "leader_stretch_unit": "inches",
        "mark_glyph_kws": {"markersize": 30},
    },
    palette=sns.color_palette()[8:],
)
og.source_axes.set_axis_off()
og.broadcast(plt.imshow, image, extent=(0, 1, 0, 1), origin="upper", zorder=-1)
otst.inset_outsets(
    og,
    insets=[  # note manual inset position/scaling as (x0, xy0, width, height)
        (0.02, 0.53, 0.50, 0.44),
        (0.05, 0.28, 0.9, 0.23),
    ],
)
og.marqueeplot()

pass

## `kdeplot`



In [ ]:
df = sns.load_dataset("penguins")

fig, ax = plt.subplots(1)

sns.kdeplot(
    data=df,
    x="body_mass_g",
    y="bill_depth_mm",
    ax=ax,
    fill=True,
    clip=((2200, 6800), (10, 25)),
    thresh=0,
    levels=100,
    cmap="rocket",
)
otst.draw_marquee(
    (5200, 6000),
    (14, 16),
    ax,
    color="teal",
    mark_glyph=otst_mark.MarkArrow(rotate_angle=-45),
    leader_stretch_unit="inches",
    leader_stretch=0.2,
    zorder=10,
)
ax.annotate("zoink?", (6300, 17), color="teal", zorder=5)
pass

In [ ]:
og = otst.OutsetGrid(
    data=sns.load_dataset("iris").dropna(),
    x="petal_width",
    y="petal_length",
    col="species",
    col_wrap=2,
    color=sns.color_palette()[1],
    marqueeplot_kws={
        "mark_glyph": otst.mark.MarkAlphabeticalBadges,
    },
    marqueeplot_source_kws={"leader_tweak": otst_tweak.TweakReflect()},
    marqueeplot_outset_kws={
        "leader_tweak": otst_tweak.TweakReflect(vertical=True)
    },
)
og.map_dataframe(
    sns.kdeplot, x="petal_width", y="petal_length", legend=False, zorder=0
)
og.map_dataframe_outset(
    sns.scatterplot,
    x="petal_width",
    y="petal_length",
    legend=False,
    zorder=0,
)
og.marqueeplot()

pass

In [ ]:
data = sns.load_dataset("iris").dropna()

og = otst.OutsetGrid(
    aspect=1.5,
    data=data,
    x="petal_width",
    y="petal_length",
    col="species",
    col_order=sorted(data["species"].unique()) + ["_1", "_2"],
    hue="species",
    marqueeplot_kws={
        "mark_glyph": otst.mark.MarkAlphabeticalBadges,
    },
    marqueeplot_source_kws={"leader_tweak": otst_tweak.TweakReflect()},
    marqueeplot_outset_kws={
        "leader_tweak": otst_tweak.TweakReflect(vertical=True)
    },
)
og.map_dataframe_source(
    sns.kdeplot,
    x="petal_width",
    y="petal_length",
    alpha=0.4,
    fill=True,
    zorder=0,
)
og.map_dataframe_source(
    sns.kdeplot,
    x="petal_width",
    y="petal_length",
    alpha=0.4,
    zorder=0,
    levels=3,
)
og.map_dataframe_outset(
    otst.patched.regplot,
    x="petal_width",
    y="petal_length",
    line_kws={"lw": 1},
    scatter_kws={"s": 1},
)
sns.move_legend(og.source_axes, loc="center left", bbox_to_anchor=(1, 0.5))
# og.source_axes.legend().set_frame_on(False)

insets = otst_util.layout_corner_insets(
    3, "NW"
) + otst_util.layout_corner_insets(2, "SE")
otst.inset_outsets(og, insets=insets)

og.outset_axes[3].set_title("petal_width", fontsize=6)
og.outset_axes[3].set_aspect("auto")
og.outset_axes[3].set_autoscale_on(True)
sns.kdeplot(
    data,
    x="petal_width",
    hue="species",
    fill=True,
    ax=og.outset_axes[3],
    legend=False,
    clip_on=False,
    bw_adjust=2,
)

og.outset_axes[4].set_title("petal_length", fontsize=6)
og.outset_axes[4].set_aspect("auto")
og.outset_axes[4].set_autoscale_on(True)
sns.kdeplot(
    data,
    x="petal_length",
    hue="species",
    fill=True,
    ax=og.outset_axes[4],
    legend=False,
    clip_on=False,
    bw_adjust=2,
)

## `lineplot`

In [ ]:
og = otst.OutsetGrid(
    aspect=2,
    data=[(210, 6, 250, 12)],
    col_wrap=1,
    x="days",
    y="profit",
)

rs = np.random.RandomState(365)
values = rs.randn(365, 4).cumsum(axis=0)
dates = np.array(range(365))
data = pd.DataFrame(values, dates, columns=["A", "B", "C", "D"])
data = data.rolling(7).mean()

og.broadcast(
    sns.lineplot,
    data=data,
    palette="tab10",
    linewidth=2.5,
    zorder=-1,
)
og.marqueeplot()
pass

## `patches`

In [ ]:
import copy
from matplotlib import patches as mpl_patches
import numpy as np
import outset as otst

# adapted from https://matplotlib.org/stable/gallery/shapes_and_collections/ellipse_demo.html

NUM = 3000

ells = [
    mpl_patches.Ellipse(
        xy=np.random.rand(2) * 20,
        width=np.random.rand(),
        height=np.random.rand(),
        angle=np.random.rand() * 360,
        facecolor=np.random.rand(3),
        alpha=0.4,
        zorder=-2,
    )
    for i in range(NUM)
]

og = otst.OutsetGrid(
    [(4, 4, 7, 6), (10, 10, 13, 12)],
    marqueeplot_source_kws=dict(
        frame_face_kws={"alpha": 0.5},
        leader_face_kws={"alpha": 1.0},
        leader_edge_kws={"ls": "-"},
    ),
)


def plot(ax):
    ax.set(xlim=(0, 20), ylim=(0, 20), aspect="equal")

    for e in ells:
        e = copy.copy(e)
        ax.add_artist(e)
        e.set_clip_box(ax.bbox)


og.broadcast(plot)
og.marqueeplot()

pass

In [ ]:
# adapted from https://matplotlib.org/stable/gallery/shapes_and_collections/ellipse_demo.html

NUM = 3000

ells = [
    mpl_patches.Ellipse(
        xy=np.random.rand(2) * 20,
        width=np.random.rand(),
        height=np.random.rand(),
        angle=np.random.rand() * 360,
        facecolor=np.random.rand(3),
        alpha=0.4,
        zorder=-2,
    )
    for i in range(NUM)
]

og = otst.OutsetGrid(
    [(4, 4, 5, 5)],
    marqueeplot_source_kws=dict(
        frame_face_kws={"alpha": 1.0, "color": "lightblue"},
        frame_inner_pad=0.3,
        leader_face_kws={"alpha": 1.0},
        leader_edge_kws={"ls": "-"},
    ),
)


def plot(ax):
    ax.set(xlim=(0, 20), ylim=(0, 20), aspect="equal")

    for e in ells:
        e = copy.copy(e)
        ax.add_artist(e)
        e.set_clip_box(ax.bbox)


og.broadcast(plot)
og.marqueeplot()

otst.inset_outsets(og, strip_ticks=False)

for ax in og.outset_axes:
    og.source_axes.add_patch(
        mpl_patches.Rectangle(
            (-0.33, -0.25),
            1.5,
            1.5,
            transform=ax.transAxes,
            facecolor="white",
            edgecolor="none",
            zorder=-1,
        )
    )

pass

## `plot`

In [ ]:
og = otst.OutsetGrid(
    aspect=1,
    data=otst_util.NamedFrames(
        peak=(2.3 / 2, 0.95, 4 / 2, 1.05),
        mid=(2.9, -0.08, 3.4, 0.08),
    ),
    marqueeplot_kws={"mark_glyph": otst_mark.MarkAlphabeticalBadges(start="X")},
    col="region",
    hue=False,
)

x = np.linspace(0, 6, 10000)

og.broadcast(
    plt.plot,
    x,
    np.sin(x) + np.cos(x * 100) * 0.01,
    color="orange",
)
og.marqueeplot()
pass

## `regplot`

In [ ]:
og = otst.OutsetGrid(
    data=sns.load_dataset("iris"),
    x="petal_length",
    y="petal_width",
    col="species",
    hue="species",
    col_wrap=2,
    marqueeplot_source_kws={
        "leader_stretch": 0.07,
        "mark_retract": 0.25,
    },
    zorder=4,
)

og.map_dataframe_source(
    sns.kdeplot,
    x="petal_length",
    y="petal_width",
    legend=False,
)
og.map_dataframe_outset(
    otst_patched.regplot,
    x="petal_length",
    y="petal_width",
)
og.marqueeplot()
og.add_legend()

pass

## `scatterplot`

In [ ]:
og = otst.OutsetGrid(
    data=[(73.5, 23.5, 78.5, 31.5)],
    color=sns.color_palette()[-1],
    marqueeplot_kws={
        "mark_glyph": otst.mark.MarkAlphabeticalBadges(start="A"),
        "frame_outer_pad": 0.2,
        "frame_outer_pad_unit": "inches",
        "frame_face_kws": {"facecolor": "none"},
    },
)
og.broadcast(
    sns.scatterplot,
    data=sns.load_dataset("mpg").dropna(),
    x="horsepower",
    y="mpg",
    hue="origin",
    size="weight",
    sizes=(40, 400),
    alpha=0.5,
    palette="muted",
    zorder=0,
)
og.marqueeplot(equalize_aspect=False)
og.add_legend()

pass

In [ ]:
plt.close("all")
og = otst.OutsetGrid(
    aspect=1.6,
    data=[
        (73.5, 23.5, 78.5, 31.5),
        (58.5, 30.5, 65.5, 38.5),
    ],
    color=sns.color_palette()[-1],
    marqueeplot_kws={
        "mark_glyph": otst.mark.MarkAlphabeticalBadges(start="A"),
        "frame_outer_pad": 0.2,
        "frame_outer_pad_unit": "inches",
        "frame_edge_kws": {"lw": 1, "ec": "k"},
        "frame_face_kws": {"facecolor": "none"},
        "leader_edge_kws": {"lw": 1, "ec": "k", "alpha": 0.5},
        "leader_face_kws": {"alpha": 0.8, "zorder": -2},
        "leader_stretch": 1,
    },
    marqueeplot_outset_kws={
        "frame_outer_pad": 0.1,
        "frame_outer_pad_unit": "axes",
        "leader_face_kws": {"alpha": 1.0},
        "leader_stretch": 0.15,
        "leader_stretch_unit": "inches",
    },
)
og.broadcast(
    sns.scatterplot,
    data=sns.load_dataset("mpg").dropna(),
    x="horsepower",
    y="mpg",
    hue="origin",
    size="weight",
    sizes=(40, 400),
    alpha=0.5,
    palette="muted",
    zorder=0,
)
sns.move_legend(og.source_axes, loc="center left", bbox_to_anchor=(1, 0.5))
otst.inset_outsets(og, equalize_aspect=False)
og.marqueeplot(equalize_aspect=False)
og.source_axes.set_title("vroom vroom\n")

pass

In [ ]:
og = otst.OutsetGrid(
    data=sns.load_dataset("penguins").dropna(),
    x="bill_length_mm",
    y="bill_depth_mm",
    col="island",
    hue="species",
    marqueeplot_kws={"mark_glyph": otst_mark.MarkRomanBadges},
    marqueeplot_source_kws={
        "leader_face_kws": {"alpha": 0.2},
        "leader_tweak": otst_tweak.TweakSpreadArea(
            spread_factor=(2, 2.5),
            xlim=(45.5, 52),
            ylim=(21, 24),
        ),
    },
)
og.map_dataframe(
    sns.scatterplot,
    x="bill_length_mm",
    y="bill_depth_mm",
)
og.marqueeplot()
og.set_axis_labels("bill length (mm)", "bill depth (mm)")
og.add_legend()

pass

## `streamplot`

In [ ]:
# adapted from # adapted from https://matplotlib.org/stable/gallery/images_contours_and_fields/plot_streamplot.html#streamplot

w = 3
Y, X = np.mgrid[-w:w:100j, -w:w:100j]
U = -1 - X**2 + Y
V = 1 + X - Y**2
speed = np.sqrt(U**2 + V**2)
args = [X, Y, U, V]

dummy_ax = plt.gca()
stream = dummy_ax.streamplot(
    *args,
    color=U,
    cmap="viridis",
)


og = otst.OutsetGrid(
    [(0.5, 1.25, 1, 1.5)],
    hue=False,
    marqueeplot_kws={"color": "fuchsia"},
    zorder=10,
)
og.broadcast(
    plt.streamplot,
    *args,
    color=U,
    linewidth=1,
    cmap="viridis",
    density=otst_util.SplitKwarg(source=2, outset=6),
    arrowstyle="->",
    arrowsize=1.5,
)
og.marqueeplot()
cbar = og.figure.colorbar(
    stream.lines,
    ax=og.axes.flat[-1],
    location="right",
    fraction=1e-9,
)
cbar.ax.set_position([1, 0.15, 0.2, 0.73])

plt.close(dummy_ax.figure)

pass

In [ ]:
# adapted from # adapted from https://matplotlib.org/stable/gallery/images_contours_and_fields/plot_streamplot.html#streamplot

w = 3
Y, X = np.mgrid[-w:w:100j, -w:w:100j]
U = -1 - X**2 + Y
V = 1 + X - Y**2
speed = np.sqrt(U**2 + V**2)
args = [X, Y, U, V]

dummy_ax = plt.gca()
stream = dummy_ax.streamplot(
    *args,
    color=U,
    cmap="viridis",
)


og = otst.OutsetGrid(
    [(0.5, 1.25, 1, 1.5)],
    hue=False,
    marqueeplot_kws={"color": "fuchsia"},
    zorder=10,
)
og.broadcast(
    plt.streamplot,
    *args,
    color=U,
    linewidth=1,
    cmap="viridis",
    density=otst_util.SplitKwarg(source=2, outset=6),
    arrowstyle="->",
    arrowsize=1.5,
)
cbar = og.figure.colorbar(
    stream.lines,
    ax=og.axes.flat[-1],
    location="right",
    fraction=1e-9,
)
og.marqueeplot()
for ax in og.outset_axes:
    ax.set_aspect("auto")
    ax.set_autoscale_on(True)

otst.inset_outsets(
    og,
    insets="SW",
    equalize_aspect=False,
)
for ax in og.outset_axes:
    for spine in ax.spines.values():
        spine.set_linewidth(6)
        spine.set_edgecolor("white")
        spine.set_visible(True)

cbar.ax.set_position([1, 0.15, 0.2, 0.73])

plt.close(dummy_ax.figure)

pass